In [1]:
import re
import nltk
import requests
import pandas as pd


In [3]:
#!pip install bs4

In [2]:
from bs4 import BeautifulSoup

In [4]:
#!pip install openpyxl

In [200]:
df = pd.read_excel('input.xlsx')
df

,URL_ID,URL
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...
...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...
97,blackassign0098,https://insights.blackcoffer.com/contribution-...
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...


In [198]:
df.loc[0, "URL"]

'https://insights.blackcoffer.com/rising-it-cities-and-its-impact-on-the-economy-environment-infrastructure-and-city-life-by-the-year-2040-2/'

In [199]:
df.loc[1, "URL"]

'https://insights.blackcoffer.com/rising-it-cities-and-their-impact-on-the-economy-environment-infrastructure-and-city-life-in-future/'

In [6]:
df.shape

(100, 2)

In [7]:
import os

In [19]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

for ix in range(df.shape[0]):

    try:
        response = requests.get(df.iloc[ix,1], headers=headers).text
        parsed_data = BeautifulSoup(response, 'lxml')
        article = parsed_data.find('article')
        if article:
            title = article.find('h1').text
            body = article.find('div', class_='td-post-content').text

            try:
                footer = article.find('div',class_='td-post-content').pre.text
                body = body.replace(footer,'')
            except Exception as e:
                print(f"Article {df.iloc[ix,0]} has no footer!")

            body = body.replace('\n',' ')
            body = body.replace('\xa0',' ')
            body = ' '.join(body.split())

        else:
            print(f"There is no article at {df.iloc[ix,0]}")
            title = ''
            body = ''

    except Exception as e:
        print(f"An error occurred at {df.iloc[ix,0]}: {e}")
        title = ''
        body = ''

    folder_path = "articles"
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        
    filename = os.path.join(folder_path, f"{df.iloc[ix,0]}.txt")
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(f"{title}\n\n")
        file.write(body)

There is no article at blackassign0036
Article blackassign0046 has no footer!
Article blackassign0047 has no footer!
There is no article at blackassign0049
Article blackassign0083 has no footer!
Article blackassign0084 has no footer!
Article blackassign0093 has no footer!


In [26]:
#import zipfile module
from zipfile import ZipFile

with ZipFile('MasterDictionary.zip', 'r') as f:
    

#extract in current directory
    f.extractall()

In [27]:
#import zipfile module
from zipfile import ZipFile

with ZipFile('StopWords.zip', 'r') as f:
    

#extract in current directory
    f.extractall()

In [34]:
# for dirname, _, filenames in os.walk('/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
#         print("hello")

In [16]:
#extract the text from the articles to dictionary

article_dics = {}

folder = "articles"

for filenm in os.listdir(folder):
    file_pt = os.path.join(folder, filenm)
    
    if os.path.isfile(file_pt):
        try:
            with open(file_pt, 'r', encoding = 'utf-8') as file: # encoding = utf-8 - specify that content are getting read by unicode formet
                                                                 # sothat make sure not losing anything like otherthan ASchii content.
                res = file.readlines()
                article_dics[int(filenm[11:15])] = res[2]
        except Exception as e:
            print(f"some excpetion:{e} on {filenm}")
            article_dics[int(filenm[11:15])] = ''
            
    

some excpetion:list index out of range on blackassign0036.txt
some excpetion:list index out of range on blackassign0049.txt


In [ ]:
# articles_sc

In [203]:
# article_dics[1]

In [60]:
import re
def extract(text):
#     text = "dsfi jf84 8bu 4bt8&^hu dhe)';"

    text = re.sub(r"[^a-z0-9A-Z]", " ", text.lower()) # remove all character except what mentioned in function
    words = text.split() # split it by space and store it as list
    specials = '@_!#$%^&*()<>?/\|}{~:;.[]' 
    word_lst = [''.join(filter(lambda char: char not in specials, word)) for word in words] # in the case of lleft any spcecial character
#     then join by single space as whole text
    return word_lst
#     return words


In [205]:
text = extract(article_dics[1])
# text

# store list of stopwords in a list

In [75]:
folders = "stopWords"
stopwords = []

for filenm in os.listdir(folders):
    file_pth = os.path.join(folders, filenm)
    
    if os.path.isfile(file_pth):
        with open(file_pth, 'r', encoding = 'latin-1') as file:
            words = file.readlines()
            words = ' '.join(words)
            stopwords += extract(words)
# stopwords

# clean stop words from articles

In [76]:
clean_articles = {}

for i, text in article_dics.items():
    words = extract(text)
    words = [word for word in words if word not in stopwords]
    clean_articles[i] = words
    

In [207]:
# print(clean_articles[90])

In [84]:
# positive_lst = []
# negative_lst = []

master_dic = {}
folder = "MasterDictionary"
for filenm in os.listdir(folder):
    file_pth = os.path.join(folder, filenm)
    
    if os.path.isfile(file_pth):
         with open(file_pth, 'r', encoding = 'latin-1') as file:
                words = file.read()
                master_dic[filenm[:8]] = words.split('\n')
    
#When you call file.read(), it reads the entire contents of the file as a single string.

#When you call file.readlines(), it reads all the lines of the file and returns them as a list of strings.


In [92]:
# master_dic['positive']

In [110]:
positive_dics = {}
negative_dics = {}

for i, lst in clean_articles.items():
#     words1 = [word for word in lst if word in master_dic['positive'] and word not in positive_dics[i]]
    if i in positive_dics:
        words1 = [word for word in lst if word in master_dic['positive'] and word not in positive_dics[i]]

    else:
        words1 = [word for word in lst if word in master_dic['positive']]
    
    positive_dics[i] = words1
    
    
    if i in negative_dics:
        words2 = [word for word in lst if word in master_dic['negative'] and word not in negative_dics[i]]

    else:
        words2 = [word for word in lst if word in master_dic['negative']]
    
    negative_dics[i] = words2
    
#     words2 = [word for word in lst if word in master_dic['negative']]
#     negative_dics[i] = words2
    
    

In [113]:
# positive_dics[2]

In [218]:

def positive_Score(lst):
    score = 0
    
    for word in lst:
        if word in master_dic['positive']:
            score += 1
    return score

In [212]:
positive_Score(clean_articles[1])

27

In [129]:
def negative_Score(lst):
    score = 0
    
    for word in lst:
        if word in master_dic['negative']:
            score += -1
    return score


In [210]:
negative_Score(clean_articles[1])

-6

In [132]:
def polarity_score(Positive_Score, Negative_Score):
    Polarity_Score = (Positive_Score - Negative_Score)/ ((Positive_Score + Negative_Score) + 0.000001)
    return Polarity_Score

In [190]:
def subjective_score(pos, neg, n):
#     len = len(clean_articles[i])
    res = ((pos + neg )/ (n+0.000001))
    return res
    

In [153]:
def syllabel(text):
    vowel = "aeiou"
    
    words = text.split()
    syllabel_cnt = []
    cnt_word_complex = 0
    
    for word in words:
        syllable = 0
        prev_char = ''
        
        for char in word:
            if char in vowel and prev_char not in vowel:
                syllable += 1
            prev_char = char
            
        if word.lower().endswith(("ed", "es")):
            syllable -= 1
        
        if syllable == 0:
            syllable = 1
        
        syllabel_cnt.append(syllable)
        
        
        if syllable > 2:
            cnt_word_complex += 1
        
    return syllabel_cnt, cnt_word_complex

In [154]:
def personal_pronoun(text):
    
    words = re.findall(r'\b(I |we|our|us)\b', text)
    US_count = re.findall(r'\b(US)\b', text)
    return len(words)-len(US_count)

In [228]:
positive_Score(clean_articles[0])

KeyError: 0

In [243]:
for i in range(1, 100):
    df.loc["id"] = int(i)
    


In [244]:
df.head()

,URL_ID,URL,POSITIVE_SCORE,NEGATIVE_SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,id
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,NaN,NaN,NaN,NaN,1.0
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,27.0,6.0,1.571428,0.045356,2.0
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,52.0,31.0,3.952381,0.031770,3.0
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,36.0,25.0,5.545454,0.018272,4.0
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,35.0,74.0,-2.794872,-0.067474,5.0


In [261]:
# p_score = []
# n_score = []

for i, lst in clean_articles.items():
    
#     df["id"] = 
    
    num_of_words = len(extract_words(article_dics[i]))

    p_score = positive_Score(lst)
    n_score = negative_Score(lst)*(-1)
    syllable_cnt, complex_word_count = syllabel(article_dics[i])
    n = len(lst)

# df.loc[row_label, column_label] accessing a particular cell and modifiying
# DataFrame.loc -> uses to access column and rows
    df.loc[df['id']== i,"POSITIVE_SCORE"]= p_score
    df.loc[df['id'] == i, "NEGATIVE_SCORE"] = n_score
#     df.loc[i, "POSITIVE_SCORE"] = p_score
#     df.loc[i, "NEGATIVE_SCORE"] = n_score
#     print(i)
    
    df.loc[df['id']== i, "POLARITY SCORE"] = polarity_score(p_score, n_score)
    df.loc[df['id']== i, "SUBJECTIVITY SCORE"] = subjective_score(p_score, n_score, n)
    

#     print(df['URL_ID'])

    avg_sen_len = num_of_words / article_dics[i].count('.') if article_dics[i].count('.') else 0
    per_of_com_words = complex_word_count / num_of_words if num_of_words else 0
    
    df.loc[df['id']== i, 'AVG SENTENCE LENGTH'] = avg_sen_len
    df.loc[df['id']== i,  'PERCENTAGE OF COMPLEX WORDS'] = per_of_com_words
    df.loc[df['id']== i,  'FOG INDEX'] = 0.4 * (avg_sen_len + per_of_com_words)
    
    df.loc[df['id']== i, 'AVG NUMBER OF WORDS PER SENTENCE'] = avg_sen_len
    df.loc[df['id']== i, 'COMPLEX WORD COUNT'] = complex_word_count
#    
    df.loc[df['id']== i,  'SYLLABLE PER WORD'] = sum(syllable_cnt)
    df.loc[df['id']== i,  'PERSONAL PRONOUNS'] = personal_pronoun(article_dics[i])
    df.loc[df['id']== i,  'AVG WORD LENGTH'] = sum([len(word) for word in extract_words(article_dics[i])]) / num_of_words if num_of_words else 0

In [184]:
# p_score
# df["POSITIVE_SCORE"] = p_score

In [263]:
df.head(5)

,URL_ID,URL,POSITIVE_SCORE,NEGATIVE_SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,id,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,27.0,6.0,0.636364,0.071274,1.0,15.525641,0.090008,6.246260,15.525641,109.0,1771.0,11.0,4.551610
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,52.0,31.0,0.253012,0.125567,2.0,18.345679,0.186406,7.412834,18.345679,277.0,2467.0,5.0,5.356662
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,36.0,25.0,0.180328,0.101329,3.0,19.250000,0.261596,7.804638,19.250000,282.0,2023.0,20.0,5.982375
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,35.0,74.0,-0.357798,0.188581,4.0,20.705882,0.257576,8.385383,20.705882,272.0,1927.0,5.0,5.892045
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,21.0,9.0,0.400000,0.090634,5.0,17.743590,0.173410,7.166800,17.743590,120.0,1121.0,6.0,5.374277


In [267]:
import nltk 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\NOMAN
[nltk_data]     KHAN\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [268]:
 
for i, text in article_dics.items():
    
    df.loc[df['id']== i, 'WORD COUNT'] = len([word for word in extract_words(text) if word not in nltk.corpus.stopwords.words('english')])

In [269]:
df.head()

,URL_ID,URL,POSITIVE_SCORE,NEGATIVE_SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,id,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH,WORD COUNT
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,27.0,6.0,0.636364,0.071274,1.0,15.525641,0.090008,6.246260,15.525641,109.0,1771.0,11.0,4.551610,633.0
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,52.0,31.0,0.253012,0.125567,2.0,18.345679,0.186406,7.412834,18.345679,277.0,2467.0,5.0,5.356662,890.0
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,36.0,25.0,0.180328,0.101329,3.0,19.250000,0.261596,7.804638,19.250000,282.0,2023.0,20.0,5.982375,680.0
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,35.0,74.0,-0.357798,0.188581,4.0,20.705882,0.257576,8.385383,20.705882,272.0,1927.0,5.0,5.892045,670.0
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,21.0,9.0,0.400000,0.090634,5.0,17.743590,0.173410,7.166800,17.743590,120.0,1121.0,6.0,5.374277,417.0


In [282]:
df = df.drop(columns = ['id'])
df

,URL_ID,URL,POSITIVE_SCORE,NEGATIVE_SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH,WORD COUNT
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,27.0,6.0,0.636364,0.071274,15.525641,0.090008,6.246260,15.525641,109.0,1771.0,11.0,4.551610,633.0
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,52.0,31.0,0.253012,0.125567,18.345679,0.186406,7.412834,18.345679,277.0,2467.0,5.0,5.356662,890.0
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,36.0,25.0,0.180328,0.101329,19.250000,0.261596,7.804638,19.250000,282.0,2023.0,20.0,5.982375,680.0
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,35.0,74.0,-0.357798,0.188581,20.705882,0.257576,8.385383,20.705882,272.0,1927.0,5.0,5.892045,670.0
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,21.0,9.0,0.400000,0.090634,17.743590,0.173410,7.166800,17.743590,120.0,1121.0,6.0,5.374277,417.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,blackassign0098,https://insights.blackcoffer.com/contribution-...,5.0,2.0,0.428571,0.035897,16.458333,0.169620,6.651181,16.458333,67.0,615.0,1.0,5.326582,257.0
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...,12.0,3.0,0.600000,0.056818,18.000000,0.086601,7.234641,18.000000,53.0,892.0,4.0,4.584967,360.0
99,blackassign0100,https://insights.blackcoffer.com/how-will-covi...,29.0,59.0,-0.340909,0.174603,31.176471,0.154717,12.532475,31.176471,164.0,1650.0,4.0,5.138679,617.0
100,NaN,NaN,29.0,59.0,-2.933333,-0.059524,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [283]:
df.to_excel('Output Data Structure.xlsx', index=False)